In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = spark.read.format("delta")\
    .options(header="true", inferSchema="true")\
    .load("abfss://bronze@netflixprojectdlba.dfs.core.windows.net/netflix_titles")

In [0]:

display(df) 

In [0]:
df=df.fillna({"duration_minutes":0,"duration_seasons":1})

In [0]:

Jul 29, 2025 (<1s)
4
4
display(df) 

In [0]:
df = df.withColumn("duration_minutes", col("duration_minutes").cast(IntegerType()))\
    .withColumn("duration_seasons", col("duration_seasons").cast(IntegerType()))

In [0]:
df.printSchema()

In [0]:
df = df.withColumn("shorttitle",split(col('title'),':')[0])

In [0]:

df.display() 

In [0]:
df = df.withColumn("rating",split(col('rating'),'-')[0]) df.display() 

In [0]:
df = df.withColumn("type_flag",when(col("type")=="Movie",1)\
    .when(col("type")=="TV Show",2)\
    .otherwise(0))
df.display()


In [0]:
from pyspark.sql.window import Window

In [0]:
df = df.withColumn("duration_ranking",dense_rank().over(Window.orderBy(col("duration_minutes").desc())))

In [0]:
display(df) 

In [0]:
df.createOrReplaceTempView("temp_view")

In [0]:
df.createOrReplaceGlobalTempView("global_view")

In [0]:
df = spark.sql("""
               select * from global_temp.global_view
               """)

In [0]:
display(df)

In [0]:
df_vis = df.groupBy("type").agg(count("*").alias("total_count")) display(df_vis) 

In [0]:
df.write.format("delta")\
    .mode("overwrite")\
    .option("path","abfss://silver@netflixprojectdlba.dfs.core.windows.net/netflix_titles")\
    .save()

In [0]:
display(df)